In [1]:
import catboost as cb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score

PREDICCION_REAL = False

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [4]:
objetivo = (entrenamiento['Stage'] == 'Closed Won').astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_object = list(entrenamiento.select_dtypes(include=['object']).columns)
if 'Stage' in columnas_object : columnas_object.remove('Stage')

In [5]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 'Closed Won').astype(int)
    test = test.drop(columns=['Stage'])

In [6]:
entrenamiento_pool = cb.Pool(entrenamiento, objetivo,cat_features=columnas_object)
test_pool = cb.Pool(test, cat_features=columnas_object)

In [17]:
model = cb.CatBoostClassifier(task_type="GPU", devices='0:1', bootstrap_type='MVS', border_count = 56615, depth = 5, early_stopping_rounds = 11, iterations = 6, l2_leaf_reg = 0.0002819635841675252, learning_rate = 0.19085900715075577, random_seed = 31486, random_strength=1.264083597622293, subsample=0.4346062744686996)
model.fit(entrenamiento_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)
print("class = ", preds)

0:	learn: 0.3091872	total: 22.5ms	remaining: 112ms
1:	learn: 0.1995144	total: 43.9ms	remaining: 87.8ms
2:	learn: 0.1494046	total: 64.3ms	remaining: 64.3ms
3:	learn: 0.1241908	total: 85.8ms	remaining: 42.9ms
4:	learn: 0.1100248	total: 107ms	remaining: 21.5ms
5:	learn: 0.1019214	total: 130ms	remaining: 0us
class =  [0 0 1 ... 1 1 1]


In [8]:
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = CatBoostClassifier(
silent=True,cat_features=columnas_object,task_type="GPU", devices='0:1', bootstrap_type='MVS'
),
search_spaces = {
        "iterations" : (5, 6),
        'learning_rate': Real(low=0.01, high=1, prior='log-uniform'),
        "random_seed" : (1,40000),
        "l2_leaf_reg" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'subsample': Real(low=0.01, high=1, prior='uniform'),
        "random_strength" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'depth': (1, 5),
        "early_stopping_rounds" : (1, 20),
        "border_count" : (1,65535)
},
scoring = 'roc_auc',
cv = StratifiedKFold(
n_splits=2,
shuffle=True,
random_state=72
),
n_jobs = 1,
n_iter = 5,
verbose = 1,
refit = True,
random_state = 72
)

In [9]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [10]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo, callback=status_print)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #1
Best ROC-AUC: 0.9911
Best params: OrderedDict([('border_count', 10204), ('depth', 5), ('early_stopping_rounds', 6), ('iterations', 5), ('l2_leaf_reg', 0.00017808806763285067), ('learning_rate', 0.21873577771989225), ('random_seed', 9750), ('random_strength', 1.6078565284271), ('subsample', 0.5979442430367322)])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #2
Best ROC-AUC: 0.9934
Best params: OrderedDict([('border_count', 56615), ('depth', 5), ('early_stopping_rounds', 11), ('iterations', 6), ('l2_leaf_reg', 0.0002819635841675252), ('learning_rate', 0.19085900715075577), ('random_seed', 31486), ('random_strength', 1.264083597622293), ('subsample', 0.4346062744686996)])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


...

Model #3
Best ROC-AUC: 0.9934
Best params: OrderedDict([('border_count', 56615), ('depth', 5), ('early_stopping_rounds', 11), ('iterations', 6), ('l2_leaf_reg', 0.0002819635841675252), ('learning_rate', 0.19085900715075577), ('random_seed', 31486), ('random_strength', 1.264083597622293), ('subsample', 0.4346062744686996)])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model #4
Best ROC-AUC: 0.9934
Best params: OrderedDict([('border_count', 56615), ('depth', 5), ('early_stopping_rounds', 11), ('iterations', 6), ('l2_leaf_reg', 0.0002819635841675252), ('learning_rate', 0.19085900715075577), ('random_seed', 31486), ('random_strength', 1.264083597622293), ('subsample', 0.4346062744686996)])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s finished


Model #5
Best ROC-AUC: 0.9934
Best params: OrderedDict([('border_count', 56615), ('depth', 5), ('early_stopping_rounds', 11), ('iterations', 6), ('l2_leaf_reg', 0.0002819635841675252), ('learning_rate', 0.19085900715075577), ('random_seed', 31486), ('random_strength', 1.264083597622293), ('subsample', 0.4346062744686996)])



In [18]:
resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series(preds)
resultados = resultados.groupby('Opportunity_ID').mean()
resultados = resultados.reset_index()
resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
resultados.to_csv("prediccion.csv", index=False)
resultados['Target'].value_counts()

1    1587
0    1055
Name: Target, dtype: int64

In [20]:
accuracy_score(test_label, preds)

0.948847420417124

In [21]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Sales_Contract_No,51.856549
1,Opportunity_Name,26.821444
2,Product_Name,9.971593
3,Product_Family,4.330494
4,Billing_Country,3.643245
5,Account_Name,2.516183
6,Territory,0.860492
7,ID,0.000000
8,Region,0.000000
9,"Pricing, Delivery_Terms_Quote_Appr",0.000000


In [22]:
preds = model.predict(entrenamiento_pool)
accuracy_score(objetivo, preds)

0.9986210253082414